# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""29048907735311…",0.983033,"""54483249738894…","""../../../../da…","""15669078359249…","""17581358970283…",false
"""83460005032982…",0.144221,"""50545605972774…","""../../../../da…","""15669078359249…","""14115905153398…",true
"""11452810122581…",0.679745,"""15224054770295…","""../../../../da…","""44747497572460…","""17581358970283…",false
"""12955024126106…",1.0,"""50545605972774…","""../../../../da…","""85714140161174…","""17581358970283…",false
"""31323781582163…",0.718982,"""54483249738894…","""../../../../da…","""85714140161174…","""22721777130126…",true
"""24097590705297…",0.0,"""15224054770295…","""../../../../da…","""10973906962754…",null,false
"""44516173293295…",0.02333,"""15224054770295…","""../../../../da…","""85714140161174…","""22721777130126…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'Customer_CLV': 'PREDICTOR_1',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'filename': 'filename',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""12490220141580…",0.983033,"""54025951998415…","""../../../../da…","""FirstMortgage3…","""18634849193355…","""Rejected"""
"""35536386632060…",0.144221,"""15473654718879…","""../../../../da…","""FirstMortgage3…","""70506948752475…","""Accepted"""
"""14929908591203…",0.679745,"""77810561528376…","""../../../../da…","""MoneyMarketSav…","""18634849193355…","""Rejected"""
"""15538361550420…",1.0,"""15473654718879…","""../../../../da…","""BasicChecking""","""18634849193355…","""Rejected"""
"""12432495103513…",0.718982,"""54025951998415…","""../../../../da…","""BasicChecking""","""16956541974060…","""Accepted"""
"""45491861437494…",0.0,"""77810561528376…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
"""12161809400743…",0.02333,"""77810561528376…","""../../../../da…","""BasicChecking""","""16956541974060…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""13547960260036…",0.983033,"""16110956871131…","""../../../../da…","""FirstMortgage3…","""66479317482720…","""Rejected"""
"""33922280322596…",0.144221,"""76274880633914…","""../../../../da…","""FirstMortgage3…","""75521597294098…","""Accepted"""
"""18041874230454…",0.679745,"""18297190530664…","""../../../../da…","""MoneyMarketSav…","""66479317482720…","""Rejected"""
"""17503810548762…",1.0,"""76274880633914…","""../../../../da…","""BasicChecking""","""66479317482720…","""Rejected"""
"""17774278417951…",0.718982,"""16110956871131…","""../../../../da…","""BasicChecking""","""10632131165560…","""Accepted"""
"""16531610994644…",0.0,"""18297190530664…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
"""18031813853145…",0.02333,"""18297190530664…","""../../../../da…","""BasicChecking""","""10632131165560…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
Customer_CLV=PREDICTOR_1
Customer_MaritalStatus=PREDICTOR_2
filename=filename
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""95733885169115…",0.983033,"""18688575405202…","""FirstMortgage3…","""63344371128538…","""Rejected"""
"""12037824036577…",0.144221,"""54121086759123…","""FirstMortgage3…","""42559481219188…","""Accepted"""
"""14672260612412…",0.679745,"""17415295875348…","""MoneyMarketSav…","""63344371128538…","""Rejected"""
"""11630534286393…",1.0,"""54121086759123…","""BasicChecking""","""63344371128538…","""Rejected"""
"""56463324866619…",0.718982,"""18688575405202…","""BasicChecking""","""11863447055019…","""Accepted"""
"""10790370548483…",0.0,"""17415295875348…","""UPlusFinPerson…",null,"""Rejected"""
"""12677543502026…",0.02333,"""17415295875348…","""BasicChecking""","""11863447055019…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""c73023dda51de1…",0.983033,"""af486bc1815c72…","""../../../../da…","""FirstMortgage3…","""2513d0e24d8d02…","""Rejected"""
"""5414b197fb87b3…",0.144221,"""3cdf7b1ae1342e…","""../../../../da…","""FirstMortgage3…","""8b55bf62fc2ecd…","""Accepted"""
"""892f5d61a9215e…",0.679745,"""806ef34eb8237f…","""../../../../da…","""MoneyMarketSav…","""2513d0e24d8d02…","""Rejected"""
"""c3aa807d9e927a…",1.0,"""3cdf7b1ae1342e…","""../../../../da…","""BasicChecking""","""2513d0e24d8d02…","""Rejected"""
"""c2d492f9a05073…",0.718982,"""af486bc1815c72…","""../../../../da…","""BasicChecking""","""1cd082ecbd5dd9…","""Accepted"""
"""b21059221605c4…",0.0,"""806ef34eb8237f…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
"""80cbe489dcecd4…",0.02333,"""806ef34eb8237f…","""../../../../da…","""BasicChecking""","""1cd082ecbd5dd9…","""Rejected"""
